In [1]:
import cv2
import mediapipe as mp

# MediaPipe Hands 솔루션 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# 손 감지 모델 설정
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 웹캠 연결
cap = cv2.VideoCapture(0)

# 랜드마크 인덱스 정의
# 손목: 0
# 중지손가락 MCP: 9
# 새끼손가락 MCP: 17
# 새끼손가락 PIP: 18

if not cap.isOpened():
    print("웹캠을 열 수 없습니다.")
    exit()

print("웹캠을 시작합니다. 'q'를 누르면 종료됩니다.")

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("프레임을 읽을 수 없습니다. 웹캠을 다시 확인해주세요.")
        break

    # 성능 향상을 위해 이미지를 읽기 전 쓰기 가능하게 설정
    image.flags.writeable = False
    # 색상 공간을 BGR에서 RGB로 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # 손 랜드마크 감지
    results = hands.process(image)

    # 랜드마크 감지 후 쓰기 가능하게 설정
    image.flags.writeable = True
    # 색상 공간을 RGB에서 BGR로 다시 변환
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 손 랜드마크 그리기 (선택 사항)
            mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS
            )

            # 원하는 랜드마크 좌표 추출 (픽셀 단위)
            image_height, image_width, _ = image.shape
            
            # 손목 (wrist) 좌표
            wrist_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x * image_width)
            wrist_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y * image_height)
            
            # 중지손가락 MCP (middle finger MCP) 좌표
            middle_finger_mcp_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].x * image_width)
            middle_finger_mcp_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].y * image_height)
            
            # 새끼손가락 MCP (pinky MCP) 좌표
            pinky_mcp_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP].x * image_width)
            pinky_mcp_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP].y * image_height)
            
            # 새끼손가락 PIP (pinky PIP) 좌표
            pinky_pip_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].x * image_width)
            pinky_pip_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].y * image_height)
            
            # 벡터 그리기
            # 1. 손목에서 중지손가락 MCP로 이어지는 벡터
            cv2.line(
                image,
                (wrist_x, wrist_y),
                (middle_finger_mcp_x, middle_finger_mcp_y),
                (0, 255, 0),  # 초록색
                2
            )
            
            # 2. 새끼손가락 MCP에서 PIP로 이어지는 벡터
            cv2.line(
                image,
                (pinky_mcp_x, pinky_mcp_y),
                (pinky_pip_x, pinky_pip_y),
                (255, 0, 0),  # 파란색
                2
            )

    # 결과 화면 표시
    cv2.imshow('Hand Landmark Vectors', image)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# 종료 시 자원 해제
hands.close()
cap.release()
cv2.destroyAllWindows()

웹캠을 시작합니다. 'q'를 누르면 종료됩니다.


In [6]:
import cv2, math, numpy as np
import mediapipe as mp

# ===== MediaPipe 설정 =====
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("웹캠을 열 수 없습니다.")
    raise SystemExit

print("웹캠 시작. 'q' 종료")

# === 각도 계산(화면 수평선 기준) + 오버레이 함수들 ===
def angle_deg_from_horizontal(p1, p2):
    """
    p1->p2 벡터가 화면 수평선과 이루는 각(도).
    OpenCV 이미지 좌표(y가 아래로 증가)이므로 atan2에 -dy 적용.
    결과: [0, 360) 범위, 반시계(↖) 방향이 +각.
    """
    dx, dy = (p2[0] - p1[0]), (p2[1] - p1[1])
    theta = math.degrees(math.atan2(-dy, dx))  # 수평선 기준
    theta = (theta + 360.0) % 360.0
    return theta

def draw_arc(img, origin, deg, radius, color, thickness=2, steps=40):
    """원점(origin)에서 0°→deg°까지 호를 그림(수평선 기준, 반시계 +)."""
    rad = math.radians(deg)
    ts = np.linspace(0.0, rad, steps)
    pts = []
    for t in ts:
        x = int(origin[0] + radius * math.cos(t))
        y = int(origin[1] - radius * math.sin(t))
        pts.append((x, y))
    if len(pts) > 1:
        cv2.polylines(img, [np.array(pts)], False, color, thickness, cv2.LINE_AA)

def annotate_angle(img, origin, target, color, label, radius=60):
    """수평선, 벡터, 각도 호, 텍스트(cos/sin)까지 한 번에 그리기."""
    # 수평선
    cv2.line(img, (0, origin[1]), (img.shape[1], origin[1]), (160,160,160), 1, cv2.LINE_AA)

    # 벡터
    cv2.arrowedLine(img, origin, target, color, 3, tipLength=0.2)

    # 각도 계산
    deg = angle_deg_from_horizontal(origin, target)
    rad = math.radians(deg)
    c, s = math.cos(rad), math.sin(rad)

    # 각도 호
    draw_arc(img, origin, deg, radius, color, 2)

    # 텍스트 위치(원점 위/아래 적절히 배치)
    tx = origin[0] + 10
    ty = origin[1] - 12 if origin[1] > 40 else origin[1] + 20
    cv2.putText(img, f"{label}: {deg:5.1f} deg  cos={c:+.2f}  sin={s:+.2f}",
                (tx, ty), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2, cv2.LINE_AA)

while cap.isOpened():
    ok, frame = cap.read()
    if not ok:
        break

    # 미러(거울) 화면 원하시면 flip(1), 필요 없으면 주석 처리
    frame = cv2.flip(frame, 1)

    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb.flags.writeable = False
    res = hands.process(rgb)
    rgb.flags.writeable = True

    if res.multi_hand_landmarks:
        for lm in res.multi_hand_landmarks:
            # (선택) 랜드마크 전체도 그리고 싶다면 주석 해제
            # mp_drawing.draw_landmarks(frame, lm, mp_hands.HAND_CONNECTIONS)

            # 픽셀 좌표
            def px(id_):
                x = int(lm.landmark[id_].x * w); y = int(lm.landmark[id_].y * h)
                return (x, y)

            WRIST = mp_hands.HandLandmark.WRIST
            MIDDLE_MCP = mp_hands.HandLandmark.MIDDLE_FINGER_MCP
            PINKY_MCP  = mp_hands.HandLandmark.PINKY_MCP
            PINKY_PIP  = mp_hands.HandLandmark.PINKY_PIP

            wrist = px(WRIST)
            middle_mcp = px(MIDDLE_MCP)
            pinky_mcp = px(PINKY_MCP)
            pinky_pip = px(PINKY_PIP)

            # 포인트 마커
            for p, col in [(wrist,(0,255,255)), (middle_mcp,(0,255,255)),
                           (pinky_mcp,(255,255,0)), (pinky_pip,(255,255,0))]:
                cv2.circle(frame, p, 5, col, -1, cv2.LINE_AA)

            # === 각도 시각화 ===
            # B: Wrist → 3rd MCP (초록)
            annotate_angle(frame, wrist, middle_mcp, (0,255,0), "Wrist→3rd MCP")

            # A: 5th MCP → PIP (파랑)
            annotate_angle(frame, pinky_mcp, pinky_pip, (255,0,0), "5th MCP→PIP")

    cv2.imshow("Angles w.r.t Horizontal (cos,sin)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

hands.close()
cap.release()
cv2.destroyAllWindows()


웹캠 시작. 'q' 종료


In [7]:
import cv2, math, numpy as np
import mediapipe as mp

# ===== MediaPipe =====
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("웹캠을 열 수 없습니다."); raise SystemExit

# ---- 각도 계산 (수평선 기준) ----
def angle_deg_from_horizontal(p1, p2):
    dx, dy = (p2[0]-p1[0]), (p2[1]-p1[1])
    theta = math.degrees(math.atan2(-dy, dx))  # OpenCV 좌표 보정
    return (theta + 360.0) % 360.0

# ---- 각도 호(arc)만 그리기(텍스트 없음) ----
def draw_arc(img, origin, deg, radius, color, thickness=3, steps=48):
    rad = math.radians(deg)
    ts = np.linspace(0.0, rad, steps)
    pts = []
    for t in ts:
        x = int(origin[0] + radius * math.cos(t))
        y = int(origin[1] - radius * math.sin(t))
        pts.append((x, y))
    if len(pts) > 1:
        cv2.polylines(img, [np.array(pts)], False, color, thickness, cv2.LINE_AA)

while True:
    ok, frame = cap.read()
    if not ok: break

    frame = cv2.flip(frame, 1)  # 필요 없으면 주석
    h, w, _ = frame.shape

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb.flags.writeable = False
    res = hands.process(rgb)
    rgb.flags.writeable = True

    if res.multi_hand_landmarks:
        lm = res.multi_hand_landmarks[0]
        def px(id_):
            return (int(lm.landmark[id_].x * w), int(lm.landmark[id_].y * h))

        WRIST = mp_hands.HandLandmark.WRIST
        M_MCP = mp_hands.HandLandmark.MIDDLE_FINGER_MCP
        P_MCP = mp_hands.HandLandmark.PINKY_MCP
        P_PIP = mp_hands.HandLandmark.PINKY_PIP

        wrist = px(WRIST)
        middle_mcp = px(M_MCP)
        pinky_mcp  = px(P_MCP)
        pinky_pip  = px(P_PIP)

        # 수평선(얇게)
        cv2.line(frame, (0, wrist[1]), (w, wrist[1]), (170,170,170), 1, cv2.LINE_AA)

        # 벡터: 초록=WRIST->3rd MCP, 파랑=5th MCP->PIP
        cv2.arrowedLine(frame, wrist,      middle_mcp, (0,255,0), 4, tipLength=0.22)
        cv2.arrowedLine(frame, pinky_mcp,  pinky_pip,  (255,0,0), 4, tipLength=0.22)

        # 각도 호(텍스트 없음)
        deg_B = angle_deg_from_horizontal(wrist,     middle_mcp)
        deg_A = angle_deg_from_horizontal(pinky_mcp, pinky_pip)
        draw_arc(frame, wrist,     deg_B, 60, (0,255,0), thickness=3)
        draw_arc(frame, pinky_mcp, deg_A, 45, (255,0,0), thickness=3)

        # 포인트 마커(작게, 필요 없으면 주석)
        for p,c in [(wrist,(0,255,255)), (middle_mcp,(0,255,255)),
                    (pinky_mcp,(255,255,0)), (pinky_pip,(255,255,0))]:
            cv2.circle(frame, p, 4, c, -1, cv2.LINE_AA)

    cv2.imshow("Angles (no text)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

hands.close()
cap.release()
cv2.destroyAllWindows()


In [3]:
import cv2, math, numpy as np
import mediapipe as mp

# ===== 설정: 화면에 보여줄 지표 선택 =====
# "angle" | "cos" | "sin"
METRIC = "angle"

# ===== MediaPipe =====
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False, max_num_hands=1,
    min_detection_confidence=0.5, min_tracking_confidence=0.5
)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("웹캠을 열 수 없습니다."); raise SystemExit

def angle_deg_from_horizontal(p1, p2):
    dx, dy = (p2[0]-p1[0]), (p2[1]-p1[1])
    th = math.degrees(math.atan2(-dy, dx))     # OpenCV 좌표 보정
    return (th + 360.0) % 360.0

def draw_arc(img, origin, deg, radius, color, thickness=3, steps=48):
    rad = math.radians(deg)
    ts = np.linspace(0.0, rad, steps)
    pts = [(int(origin[0] + radius*math.cos(t)),
            int(origin[1] - radius*math.sin(t))) for t in ts]
    if len(pts) > 1:
        cv2.polylines(img, [np.array(pts)], False, color, thickness, cv2.LINE_AA)

def put_metric(img, origin, deg, color):
    rad = math.radians(deg)
    val = {"angle": f"{deg:5.1f} deg", 
           "cos": f"cos={math.cos(rad):+.2f}",
           "sin": f"sin={math.sin(rad):+.2f}"}[METRIC]
    x = min(origin[0] + 10, img.shape[1]-180)
    y = origin[1] - 12 if origin[1] > 40 else origin[1] + 22
    cv2.putText(img, val, (x, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.65, color, 2, cv2.LINE_AA)

while True:
    ok, frame = cap.read()
    if not ok: break
    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb.flags.writeable = False
    res = hands.process(rgb)
    rgb.flags.writeable = True

    if res.multi_hand_landmarks:
        lm = res.multi_hand_landmarks[0]
        def px(i): return (int(lm.landmark[i].x*w), int(lm.landmark[i].y*h))

        WRIST = mp_hands.HandLandmark.WRIST
        M_MCP = mp_hands.HandLandmark.MIDDLE_FINGER_MCP
        P_MCP = mp_hands.HandLandmark.PINKY_MCP
        P_PIP = mp_hands.HandLandmark.PINKY_PIP

        wrist, middle_mcp = px(WRIST), px(M_MCP)
        pinky_mcp, pinky_pip = px(P_MCP), px(P_PIP)

        # 수평선 2개(흰색)
        cv2.line(frame, (0, wrist[1]),     (w, wrist[1]),     (255,255,255), 2, cv2.LINE_AA)
        cv2.line(frame, (0, pinky_mcp[1]), (w, pinky_mcp[1]), (255,255,255), 2, cv2.LINE_AA)

        # 벡터: 빨강=WRIST->3rd MCP, 파랑=5th MCP->PIP
        cv2.arrowedLine(frame, wrist,     middle_mcp, (0,0,255), 4, tipLength=0.22)
        cv2.arrowedLine(frame, pinky_mcp, pinky_pip,  (255,255,0), 4, tipLength=0.22)

        # 각도 & 호
        deg_B = angle_deg_from_horizontal(wrist,     middle_mcp)
        deg_A = angle_deg_from_horizontal(pinky_mcp, pinky_pip)
        draw_arc(frame, wrist,     deg_B, 60, (0,0,255), 3)
        draw_arc(frame, pinky_mcp, deg_A, 45, (255,255,0), 3)

        # === 선택한 지표만 표시 ===
        put_metric(frame, wrist, deg_B, (0,0,255))     # 빨강
        put_metric(frame, pinky_mcp, deg_A, (255,255,0)) # 파랑

    # 상단에 현재 지표 안내
    cv2.putText(frame, f"Metric: {METRIC.upper()}   (press q to quit)",
                (12, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)

    cv2.imshow("Angle / Cos / Sin (single metric)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

hands.close(); cap.release(); cv2.destroyAllWindows()
